<p><font size="6" color='grey'> <b>
Machine Learning
</b></font> </br></p>
<p><font size="5" color='grey'> <b>
Neural Network - Keras - Cancer
</b></font> </br></p>

---


# 0  | Install & Import
***

In [ ]:
# Install


In [ ]:
# Import
from pandas import read_csv, DataFrame, concat

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

from keras.layers import Input, Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import set_random_seed, plot_model
from keras.utils import to_categorical


from tensorflow import keras
from tensorflow.config.experimental import enable_op_determinism
import tensorflow as tf

import plotly.express as px
import plotly.subplots as sp

In [ ]:
# Warnung ausstellen
import warnings
warnings.filterwarnings("ignore")

# 1 | Understand
---


<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Aufgabe verstehen</br>
✅ Daten sammeln</br>
✅ Statistische Analyse (Min, Max, Mean, Korrelation, ...)</br>
✅ Datenvisualisierung (Streudiagramm, Box-Plot, ...)</br>
✅ Prepare Schritte festlegen</br>

<p><font color='black' size="5">
📒 Anwendungsfall
</font></p>




**Beschreibung:**   
Diese Arbeit entstand aus dem Wunsch, Gewebeproben ausschließlich auf der Grundlage einer Feinnadelpunktion (FNA) genau zu diagnostizieren. In Zusammenarbeit mit Prof. Mangasarian und zwei seiner Doktoranden, Rudy Setiono und Kristin Bennett , wurde mithilfe der Multisurface-Methode (MSM) zur Mustertrennung dieser neun Merkmale ein Klassifikator erstellt, der 97 % der neuen Fälle erfolgreich diagnostizierte. Der resultierende Datensatz ist als Wisconsin Breast Cancer Data bekannt.


Die Arbeit an der Bildanalyse begann 1990 mit der Aufnahme von Nick Street in das Forschungsteam. Ziel war es, die Probe anhand eines digitalen Bildes eines kleinen Abschnitts des FNA-Objektträgers zu diagnostizieren.

**Diagnoseablauf:**

Es wird aus dem Gewebe eine FNA entnommen. Dieses Material wird dann auf einen Objektträger montiert und gefärbt, um die Zellkerne hervorzuheben. Ein Teil des Objektträgers, in dem die Zellen gut differenziert sind, wird dann mit einer Digitalkamera und einem Framegrabber-Board gescannt.
Anschließend isoliert der Anwender die einzelnen Zellkerne . Mit einem Mauszeiger zeichnet der Benutzer die ungefähre Grenze jedes Kerns. Mithilfe eines Computer-Vision-Ansatzes, konvergieren diese Annäherungen dann an die genauen nuklearen Grenzen. Sobald alle (oder die meisten) Kerne auf diese Weise isoliert wurden, berechnet das Programm Werte für jedes der zehn Merkmale jedes Kerns und misst Größe, Form und Textur. Der Mittelwert, der Standardfehler und die Extremwerte dieser Merkmale werden berechnet, was zu insgesamt 30 Kernmerkmalen für jede Probe führt.

[DataSet](https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original)

[Info](https://pages.cs.wisc.edu/~olvi/uwmp/cancer.html)

**Wertebereich: 1 – 10**      
+ 1 = eher unauffällig / typisch gutartig        

+ 10 = stark auffällig / typisch bösartig



**Features:**


+ Dicke: 1 - 10
+ Einheitlichkeit der Zellgröße: 1 - 10
+ Gleichmäßigkeit der Zellform: 1 - 10
+ Randhaftung: 1 - 10
+ Größe einzelner Epithelzellen: 1 - 10
+ Nackte Kerne: 1 - 10
+ Blandes Chromatin: 1 - 10
+ Normale Nukleolen: 1 - 10
+ Mitosen: 1 - 10

**Klassen:**

+ Klasse: (2 für gutartig, 4 für bösartig)

**Kurzerläuterung:**

| **Merkmal**                           | **Erläuterung (einfach und verständlich)**                                                                            |
| ------------------------------------- | --------------------------------------------------------------------------------------------------------------------- |
| **Dicke** *(Clump Thickness)*         | Wie dick oder dicht gepackt die Zellverbände sind – dicker = potenziell auffälliger                                   |
| **Einheitlichkeit der Zellgröße**     | Wie ähnlich oder unterschiedlich die Zellgrößen sind – starke Unterschiede können auf Bösartigkeit hinweisen          |
| **Gleichmäßigkeit der Zellform**      | Wie regelmäßig die Zellformen sind – unregelmäßige Formen sind oft verdächtig                                         |
| **Randhaftung** *(Marginal Adhesion)* | Wie gut haften die Zellen aneinander – geringe Haftung kann auf bösartige Zellen hindeuten                            |
| **Größe einzelner Epithelzellen**     | Durchschnittliche Größe bestimmter Zellen – ungewöhnlich große Zellen können ein Risikozeichen sein                   |
| **Nackte Kerne** *(Bare Nuclei)*      | Zellkerne, die außerhalb von Zellen sichtbar sind – in größerer Zahl ein typisches Merkmal bösartiger Tumoren         |
| **Blandes Chromatin**                 | Wie gleichmäßig die DNA-Struktur im Zellkern aussieht – „bland“ heißt einheitlich, Abweichungen können auffällig sein |
| **Normale Nukleolen**                 | Nukleolen = Zellkernbestandteile – viele oder große Nukleolen sind oft in bösartigen Zellen zu finden                 |
| **Mitosen**                           | Zellteilungen – viele Mitosen deuten auf schnelles Zellwachstum hin, was ein Warnzeichen sein kann                    |


In [ ]:
import pandas as pd
df = read_csv('https://raw.githubusercontent.com/ralf-42/ML_Intro/main/02_daten/05_tabellen/breast_cancer_wisconsin.csv')

In [ ]:
data = df.copy()
target = data.pop("Class")

# 2 | Prepare

---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Nicht benötigte Features löschen</br>
✅ Datentyp ermitteln/ändern</br>
✅ Duplikate ermitteln/löschen</br>
✅ Missing Values behandeln</br>
✅ Ausreißer behandeln</br>
✅ Kategorischer Features Kodieren</br>
✅ Numerischer Features skalieren</br>
✅ Feature-Engineering (neue Features schaffen)</br>
✅ Dimensionalität reduzieren</br>
✅ Resampling (Over-/Undersampling)</br>
✅ Pipeline erstellen/konfigurieren</br>
✅ Train-Test-Split durchführen</br>

<p><font color='black' size="5">
Datentyp
</font></p>

In [ ]:
all_col = data.columns
num_col = data.select_dtypes(include="number").columns
cat_col = data.select_dtypes(exclude="number").columns

<p><font color='black' size="5">
Missing Values
</font></p>

In [ ]:
data = data.dropna()

In [ ]:
target = target.loc[data.index]

<p><font color='black' size="5">
Kodieren Target auf 0/1
</font></p>

In [ ]:
target.replace([2, 4], [0, 1], inplace=True)

<p><font color='black' size="5">
Skalieren
</font></p>

In [ ]:
data[num_col] = MinMaxScaler().fit_transform(data[num_col])

<p><font color='black' size="5">
Kategorien für Wahrscheinlichkeiten
</font></p>

In [ ]:
num_categories = 2
target = to_categorical(target, num_categories)

<p><font color='black' size="5">
Train-Test-Split
</font></p>


In [ ]:
data_train, data_test, target_train, target_test = train_test_split(
    data, target, test_size=0.3, random_state=42, stratify=target
)
data_train.shape, data_test.shape, target_train.shape, target_test.shape

# 3 | Modeling
---

<p><font color='black' size="5">
Zufallszahl initialisieren
</font></p>

In [ ]:
set_random_seed(42)
enable_op_determinism()

<p><font color='black' size="5">
Aufbau des Neuronale Netzes
</font></p>


In [ ]:
model = Sequential()
model.add(Input(shape=(9,)))  # Expliziter Input Layer
model.add(Dense(20, activation="relu"))  # Hidden Layer
model.add(Dense(2, activation="sigmoid"))  # Output Layer

In [ ]:
# Visualisierung neuronales Netz
model.summary()

In [ ]:
# Visualisierung neuronales Netz
plot_model(
    model,
    to_file="nn_structure.png",
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    dpi=100,
    expand_nested=True,
    show_layer_activations=True
)

In [ ]:
# Anzahl Parameter je Layer
for layer in model.layers:
    print(f"{layer.name}: {layer.count_params()} Parameter")

<p><font color='black' size="5">
Compile
</font></p>



In [ ]:
model.compile(optimizer="adam", loss="BinaryCrossentropy", metrics=["accuracy"])

<p><font color='black' size="5">
Callback
</font></p>

In [ ]:
# early = EarlyStopping(monitor="val_loss", patience=2)

# reduce_lr = ReduceLROnPlateau(
#     monitor="val_loss",
#     factor=0.1,
#     patience=5,
#     verbose=1,
#     mode="auto",
#     min_delta=0.0001,
#     cooldown=0,
#     min_lr=0,
# )

# check = ModelCheckpoint(
#     filepath="model.keras", monitor="val_accuracy", save_best_only=True
# )

<p><font color='black' size="5">
Training
</font></p>


In [ ]:
model.fit(
    data_train,
    target_train,
    validation_split=0.2,
    epochs=100,
    # callbacks=[check, reduce_lr, check],
)

In [ ]:
print(model.history.params)
print(model.history.history.keys())
save_history = model.history.history.keys()

<p><font color='black' size="5">
Loss-Entwickung
</font></p>

In [ ]:
title_ = "Loss-Entwicklung"
px.line(
    y=model.history.history["loss"],
    title=title_,
    labels={"x": "Epochen", "y": "Loss-Wert"},
    width=800,
    height=400,
)

# 4 | Evaluate
---

[Modellperformance](https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data)

<p><font color='black' size="5">
🔭 Prognose
</font></p>

In [ ]:
target_train_pred = model.predict(data_train)
target_test_pred = model.predict(data_test)

<p><font color='black' size="5">
📱 Confusion Matrix
</font></p>

In [ ]:
# Klasse mit höchster Wahrscheinlichkeit wird auf 1 gesetzt, übrige Klassen 0
target_train_pred_ = pd.DataFrame(target_train_pred).copy()
target_train_ = pd.DataFrame(target_train).copy()
max_cols_train_pred = target_train_pred_.idxmax(axis=1)
max_cols_train = target_train_.idxmax(axis=1)

In [ ]:
# Klasse mit höchster Wahrscheinlichkeit wird auf 1 gesetzt, übrige Klassen 0
target_test_pred_ = pd.DataFrame(target_test_pred).copy()
target_test_ = pd.DataFrame(target_test).copy()
max_cols_test_pred = target_test_pred_.idxmax(axis=1)
max_cols_test = target_test_.idxmax(axis=1)

In [ ]:
target_test = pd.DataFrame(target_test)
display_labels = ['gutartig', 'bösartig']
conf_matrix = confusion_matrix(max_cols_test, max_cols_test_pred)

In [ ]:
disp = ConfusionMatrixDisplay(conf_matrix, display_labels=display_labels)
disp.plot(cmap='Blues')

In [ ]:
print(classification_report(max_cols_test, max_cols_test_pred, target_names=display_labels))

<p><font color='black' size="5">
🎯 Accuracy
</font></p>

In [ ]:
acc_train_sklearn = accuracy_score(max_cols_train, max_cols_train_pred)
print(f'Accuracy Test Data: {acc_train_sklearn * 100:.2f}%')

In [ ]:
acc_train_sklearn = accuracy_score(max_cols_test, max_cols_test_pred)
print(f'Accuracy Test Data: {acc_train_sklearn * 100:.2f}%')

<p><font color='black' size="5">
Aufbau Analysewürfel
</font></p>

In [ ]:
# Übernahme der Testdaten
cube = data_test.copy()
cube.reset_index(inplace=True)

# Übernahem Target real & predict
cube["real"] = DataFrame(max_cols_test.values, columns=["real"])
cube["predict"] = DataFrame(max_cols_test_pred, columns=["predict"])

In [ ]:
# Erstellung 2D Features über Dimensionsreduktion PCA - unsupervised
pca = PCA(n_components=2)
pca = pca.fit_transform(data_test)
pca_df = DataFrame(pca)

# Cube um pca erweitern
cube["PCA1"] = pca_df[0]
cube["PCA2"] = pca_df[1]

<p><font color='black' size="5">
Visualisierung real vs predict
</font></p>

In [ ]:
# Histogramm
title_ = "Histogramm real vs predict"
fig = px.histogram(cube, x=["real", "predict"], nbins=2, text_auto=".2s", title=title_)
fig.update_layout(barmode="group", bargap=0.1, width=600, height=600)
fig.show()

In [ ]:
# 2 x Scatterplots

cube["real_cat"] = cube["real"].astype(str)
cube["predict_cat"] = cube["predict"].astype(str)

# Farbzuteilung für Kategoriewerte
color_mapping = {"0.0": "red", "1.0": "blue"}

title_ = "Streupunktdiagramm real"
img1 = px.scatter(
    cube,
    x="PCA1",
    y="PCA2",
    color="real_cat",
    color_discrete_map=color_mapping,
    width=600,
    height=600,
)

title_ = "Streupunktdiagramm predict"
img2 = px.scatter(
    cube,
    x="PCA1",
    y="PCA2",
    color="predict_cat",
    color_discrete_map=color_mapping,
    width=600,
    height=600,
)

fig = sp.make_subplots(
    rows=1, cols=2, subplot_titles=("Scatterplot real", "Scatterplot predict")
)

for trace in img1.data:
    fig.add_trace(trace, row=1, col=1)

for trace in img2.data:
    fig.add_trace(trace, row=1, col=2)

# Layout anpassen
fig.update_layout(width=1000, height=500, title_text=title_)

# Plot anzeigen
fig.show()

In [ ]:
# real <> predict
cube[cube.real != cube.predict].describe().T

In [ ]:
cube[cube.real != cube.predict]

# 5 | Deploy
---

<p><font color='black' size="5">📋 Checkliste</font></p>

✅ Modellexport und -speicherung</br>
✅ Abhängigkeiten und Umgebung</br>
✅ Sicherheit und Datenschutz</br>
✅ In die Produktion integrieren</br>
✅ Tests und Validierung</br>
✅ Dokumentation & Wartung</br>